# 데이터 크롤링

- [awesome-devblog : feeds](https://awesome-devblog.now.sh/api/korean/people/feeds)

In [31]:
import requests, json
import os, re, csv
import pandas as pd
from tqdm import trange

In [13]:
DATA_PATH = './data'
DATA_URL = 'https://awesome-devblog.now.sh/api/korean/people/feeds'
DOCUMENTS_PATH = './data/documents.tsv'

In [137]:
def getTotal():
    """
    전체 문서 개수 조회
    """
    res = requests.get(DATA_URL, { 'size': 1 }).json()
    return res['total'][0]['count']

def getDoc(page, size):
    """
    문서 조회
    """
    page += 1
    params = {
        'sort': 'date.asc',
        'page': page,
        'size': size
    }
    return requests.get(DATA_URL, params).json()

def getDocs(size, start_page=0):
    """
    전체 문서 조회
    """
    total = getTotal()
    if size > 5000: size = 5000
    total_req = round(total/size + 0.5)
    docs = None
    for i in trange(start_page, total_req):
        doc = getDoc(i, size)
        if docs == None:
            docs = doc
        else:
            docs['data'] += doc['data']
    return docs

def updateDocs():
    """
    신규 문서 추가
    """
    size = 5000
    # imgUr(key 자체가 경우가 존재), postImage 배제
    keys = ['_id', 'title', 'link', 'date', 'description', 'author', 'tags', 'count']
    if not os.path.isfile(DOCUMENTS_PATH):
        # 데이터가 없는 경우, 전체 데이터를 불러서 신규 생성
        docs = getDocs(size)
        with open(DOCUMENTS_PATH, 'w') as tsv_file:
            tsv_writer = csv.writer(tsv_file, delimiter='\t')
            tsv_writer.writerow(['_no', '_label'] + keys)
            for i, doc in enumerate(docs['data']):
                tsv_writer.writerow([i + 1, ''] + [re.sub('\t', '', str(doc[k])) for k in keys])
    else:
        # 기존 데이터가 있는 경우, 신규 데이터를 추가
        tsv = pd.read_csv(DOCUMENTS_PATH, delimiter='\t')
        old_total = tsv.tail(1)['_no'].values[0]
        docs = getDocs(size, old_total // size)
        no = old_total + 1
        with open(DOCUMENTS_PATH, 'a') as tsv_file:
            tsv_writer = csv.writer(tsv_file, delimiter='\t')
            for i, doc in enumerate(docs['data']):
                if doc['_id'] not in tsv._id.unique():
                    tsv_writer.writerow([no + i, ''] + [re.sub('\t', '', str(doc[k])) for k in keys])

def tempSetLabel():
    """
    기존 라벨링한거 기존꺼에 반영해두기...추가하기(1회용)
    """
    return

In [138]:
updateDocs()

100%|██████████| 8/8 [00:23<00:00,  2.88s/it]


### Convert to TSV
- title, description, tag만 기록
- title이나 description이 3글자 미만인 경우 제외
- label 컬럼 추가(default 1)
 - 0 : IT 관련 글 아님
 - 1 : 완전 IT 글
 - 2 : 애매함..(향후 카테고리를 더 나누기 위한 글 : 알고리즘 / 조직, 문화 / 디자인)

In [136]:
ntsv = pd.read_csv(DOCUMENTS_PATH, delimiter='\t')
btsv = pd.read_csv('/Users/nero/Project/DocClassification/data/10000_data.tsv', delimiter='\t')

In [139]:
btsv

,Unnamed: 0,label,sentence
0,0,1,todo 리스트를 1달간 사용해본 후기 왜 todo 리스트를 사용했나요 위 질문에 ...
1,1,0,양평 블룸비스타 콩순이 패밀리룸이 궁금하신분에게 드리는 후기 늦은 여름 휴가로 양평...
2,2,1,javascript js 자바스크립트 문자열 메소드 javascript 문자열 메소...
3,3,0,우리 시장에는 어떤 교육이 필요할까요 1 시장적합형 교육과 기술적합형 교육 며칠전 ...
4,4,0,저 많은 국민은행 앱들은 대체 다 어디다 쓰는 걸까 대략 정리 구글플래이 스토어에서...
...,...,...,...
9995,9995,1,nginx rest kubernetes 164 kubernetes nginx ing...
9996,9996,1,git https javascript html web ui modal design ...
9997,9997,1,spring boot spring spring boot오류 처리에 대해 서론오류 처...
9998,9998,1,http3 http3 httpoverquic quic은 quick udp inter...


In [87]:
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

In [88]:
with open(POSTS_JSON_PATH, 'w') as outfile:
    json.dump(posts, outfile)

In [112]:
with open(POSTS_TSV_PATH, 'w') as f:
    # title, description, tags만 수집
    f.write('\t'.join(['label', 'title', 'description', 'tags', 'link']))
    f.write('\n')
    
    for post in posts:
        values = []
        if len(post['title']) < 3: continue
        if len(post['description']) < 3: continue
        if len(post['link']) < 10: continue
        title = re.sub('\t', '', post['title'])
        description = re.sub('\t', '', post['description'])
        tags = re.sub('\t', '', ' '.join(post['tags']))
        link = re.sub('\t', '', post['link'])
        f.write('\t'.join(['1', title, description, tags, link]))
        f.write('\n')
    print('done')

done
